<b>Audio and Music Processing Lab - Module 2</b><br>Rafael Caro Repetto<br>rafael.caro@upf.edu<br>15.02.2023
## AMPLab2 - Introduction to music21 (3)
This notebook presents two new functionalites of music21.

In [ ]:
from music21 import *

### Retrieving elements by offset

Let's load a score and retrieve its vocal and instrumental parts

In [ ]:
s = converter.parse('lseh-YiLunMing-WenZhaoGuan-1.xml')
pv = s.parts[0] # Vocal part
pi = s.parts[1] # Instrumental part

Now, let's retrieve just the melodic phrase used for singing the lyrics "窗前."
<br/>
In order to do that, let's find the offsets of the first and last notes sung in that melodic phrase. Note that the second character ("前.") is sung with many notes. So, in order to retrieve all the notes used for singing that character, we can find the offset of the previous note to the next sung lyric ("愁").

In [ ]:
a = '窗'
b = '愁' 
phraseBoundaries = {}
nn = pv.flat.notes.stream()
for n in nn:
    if n.lyric:
        if n.lyric == a:
            phraseBoundaries['start'] = n.offset # NOTE: .offset
        elif n.lyric == b:
            phraseBoundaries['end'] = n.previous().offset

print('Start offset:', phraseBoundaries['start'])
print('End offset:', phraseBoundaries['end'])

The method `.getElementsByOffset()` is thought for retrieving elements according to their offsets. If just one offset is input, it retrieves the element(s) at that offset.

In [ ]:
section = nn.getElementsByOffset(73).stream() # NOTE: .getElementsByOffset with one argument
print(section.elements)
firstNote = section[0]
print('This note is {} with lyric {}'.format(firstNote.nameWithOctave, firstNote.lyric))

In [ ]:
section = nn.getElementsByOffset(94).stream()
print(section.elements)
firstNote = section[0]
print('This note is {} with lyric {}'.format(firstNote.nameWithOctave, firstNote.lyric))

If two offsets are input to the `.getElementsByOffset()` method, all the elements between those two offsets are retrieved.

In [ ]:
section = nn.getElementsByOffset(73, 94).stream() # NOTE: .getElementsByOffset with two argument
section.show()

Since we retrieved notes from the stream `nn` we created, and the start offset is 73, extra measures are created to fill the gap until that offset.
<br/>
To avoid that, we can instead retrieve measures from the part stream `pv`.

In [ ]:
section = pv.getElementsByOffset(phraseBoundaries['start'], phraseBoundaries['end']).stream()
section.show()

Now we didn't get the extra measures, but we are missing the first one of our melodic phrase. Why is that?
<br/>
Let's check the documentation for the `.getElementsByOffset()` method.

In [ ]:
pv.getElementsByOffset?

It seems that we need to set the `mustBeginInSpan` parameter to `False`.

In [ ]:
section = pv.getElementsByOffset(phraseBoundaries['start'], phraseBoundaries['end'], mustBeginInSpan=False).stream()
section.show()

### Analytical tools

Music21 brings many analytical tools, that can be found in its [documentation](https://web.mit.edu/music21/doc/usersGuide/usersGuide_22_graphing.html). However, you should understand them well in order to know what they really represent.

Let's create a simple score using `tinyNotation`.

In [ ]:
s = converter.parse('tinyNotation: G16 G G G c1')
s.show()

Now, let's use music21 analytical tools to create a pitch histogram. It's as simple as this:

In [ ]:
s.plot('histogram', 'pitch')

Does it really represent what we need? Otherwise, we would like to create our own histogram

In [ ]:
import matplotlib.pyplot as plt

nn = s.flat.notes.stream()

pitchHistogram = {}

for n in nn:
    pitchHistogram[n.nameWithOctave] = pitchHistogram.get(n.nameWithOctave, 0) + n.quarterLength # NOTE: .quarterLength
    
xValues = pitchHistogram.keys()
yValues = pitchHistogram.values()

plt.bar(xValues, yValues)
plt.show()